In [10]:
from datetime import datetime
import pandas as pd
import pandas_datareader as pdr
end = datetime.now()
start = datetime(end.year - 1, end.month, end.day)
#DataReader method name is case sensitive
df = pdr.DataReader("goog", 'yahoo', start, end)
df = df.reset_index()
df = df.round(2)
df

,Date,High,Low,Open,Close,Volume,Adj Close
0,2021-08-16,138.99,136.17,138.00,138.92,18040000.0,138.92
1,2021-08-17,138.72,136.79,138.19,137.30,21272000.0,137.30
2,2021-08-18,138.29,136.42,137.12,136.57,14934000.0,136.57
3,2021-08-19,137.45,135.36,135.47,136.91,18296000.0,136.91
4,2021-08-20,138.61,136.47,137.08,138.44,15564000.0,138.44
...,...,...,...,...,...,...,...
248,2022-08-10,121.78,119.36,119.59,120.65,20497000.0,120.65
249,2022-08-11,122.34,119.55,122.08,119.82,16671600.0,119.82
250,2022-08-12,122.65,120.40,121.16,122.65,16106600.0,122.65
251,2022-08-15,123.26,121.57,122.21,122.88,15525000.0,122.88


In [25]:
import pandas as pd
from sqlalchemy import create_engine
db_host = "204.15.72.120"
db_user = "root"
db_password = "lemon123"
db_database = "final"
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=db_host, db=db_database, user=db_user, pw=db_password))
dbConnection = engine.connect()
yesterday = datetime.today()+timedelta(-1)
yesterday_format = yesterday.strftime('%Y-%m-%d')
stockname = pd.read_sql(f"select distinct stockAbbrName from stock_follow_table", dbConnection)
compamyabbreviations = stockname['stockAbbrName'].values.tolist()
#print(yesterday_format)
for name in compamyabbreviations:
    last_record_date = pd.read_sql(f"select Date from {name} ORDER BY Date DESC LIMIT 1", dbConnection)
    last_record_date = last_record_date['Date'].to_string(index = False)
    #last_record_date = last_record_date[0].strftime('%Y-%m-%d')
    #last_record_date_formant = last_record_date.strftime('%Y-%m-%d')
    if last_record_date == yesterday_format:
        print(last_record_date)

2022-08-17
2022-08-17


In [26]:
from sqlalchemy import create_engine
import pandas as pd
db_host = "localhost"
db_user = "root"
db_password = "1207"
db_database = "damg7245"
start_date = '07/01/2018'
end_date = '07/02/2018'
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
        .format(host=db_host, db=db_database, user=db_user, pw=db_password))
dbConnection    = engine.connect()
df = pd.read_sql(f"SELECT * FROM goog_stock WHERE Date BETWEEN {start_date} AND {end_date}", dbConnection)
data = df.to_dict()

data

{'Date': {},
 'High': {},
 'Low': {},
 'Open': {},
 'Close': {},
 'Volume': {},
 'Adj Close': {}}

In [ ]:
from pandas_datareader.data import DataReader
import numpy as np
df = DataReader('AAPL', data_source='yahoo', start='2012-01-01', end=datetime.now())
# Create a new dataframe with only the 'Close column 
data = df.filter(['Close'])
# Convert the dataframe to a numpy array
dataset = data.values
# Get the number of rows to train the model on
training_data_len = len(dataset) * .95

# Scale the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)\

# Create the training data set 
# Create the scaled training data set
train_data = scaled_data
# Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 61:
        print(x_train)
        print(y_train)
        print()
        
# Convert the x_train and y_train to numpy arrays 
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
# x_train.shape

from keras.models import Sequential
from keras.layers import Dense, LSTM

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)

# Create the testing data set
# Create a new array containing scaled values from index 1543 to 2002 
test_data = scaled_data[training_data_len - 60: , :]
# Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    
# Convert the data to a numpy array
x_test = np.array(x_test)

# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

# Get the models predicted price values 
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)